# Tutorial for Model2Mass

In this tutorial, we will present how to use the package Model2Mass. We will use PyDiscrete to choose a discrete finite group, explain how the model should be written for model2mass to read it, extract the lagangian, the mass matrices, and finally make a fit using flavorPy (https://github.com/FlavorPy/FlavorPy/tree/master).

Let's start by importing the relevant packages

In [ ]:
from FlavorBuilder.PyDiscrete import Group 
from FlavorBuilder.Model2Mass import Model2Mass
import flavorpy.modelfitting as mf

import pandas as pd
import sympy as sp
import numpy as np
import json

from IPython.display import display, Latex
import re


Let's start by using choosing a group. In this case, we choose A4 with GAP ID [12,3] . For details on how to use PyDiscrete go to the respective tutorial

In [ ]:
gap_path = "/users/.../gap-4.11.1/gap" #your path
groupA4 = Group(12, 3, gap_path, compute_ALL_CGC =  True)

We now import a model found by Amber (https://github.com/jake-rudolph-1/models-by-amber/blob/main/neutrino_flavor/A4_Z4/model_A4Z4_0001)

In [ ]:
modelByAmber_file = '../ModelsInPaper/A4XZ4tab2.txt'

with open(modelByAmber_file, "rb") as f:
    modelByAmber = json.load(f)

Let's take a look at the model file.

In [ ]:
modelByAmber

It is a dictionary containing different features of the model. The "particles" key tells Model2Mass the symbols you want it to assign to the different particles. For instance, here it has: 

1. $L_i$ for $i = 1,2,3 $ which represent the $\mathrm{SU}(2)_{L}$ lepton doublets
2. $E_i$ for $i = 1,2,3 $ which represent the $\mathrm{SU}(2)_{L}$ lepton singlets
3. $N_i$ for $i = 1,2,3 $ which represent the right-handed neutrino SM gauge group singlets
4. $H_u$ the up Higgs in the MSSM
5. $H_d$ the down Higgs in the MSSM
6. $\phi_i$ 5 flavons which are neutral under the SM gauge group


The "representations" key tells the representation in which each particle belongs. The ordering of the "representations" arrays should be the same as the one of "particles". In this case, A4 has four irreps. So, the first four entries of each array in the "representations" is a one-hot encoded vector that tells which irrep of A4 this particle belongs. The entries afterwards correspond to the ZN charge value.

The ZN dimension is specified in "ZN_symmetries" key. In this case, it corresponds to a Z4. This could in principle be an array with M numbers which would correspond to M $\mathbb{Z}_{N_{i}}$ with $i=1, \cdots, M$ finite abelain groups. 

The convention of the $\mathbb{Z}_{N}$ charges $q$ such that the value of this charge ranges from 1 to N. The actual charge corresponds to $q-1$. So, for instance, the charge of the Lagrangian term $L_1 E_1 H_d$ should be
$$ (q_{L_{1}} - 1 ) + ( q_{E_{1}} - 1) + (q_{H_{d}} -1 ) \,\,\,\mathrm{mod} N$$

If the corresponding array in "representations" for a given particle consists of all zeros, then this particle is assumed to not be present in the model. This is the case, if for example, you don't want to include the flavon 5 $\phi_5$.

For example, in this model, $L_{2}$ is a triplet under $A_{4}$ and has a charge of $1$ under $\mathbb{Z}_4$.

Of course, since $L_{2}$ is in a triplet, model2mass needs to know what are the other components of that triplet. The "associations" key in the dictionary model tells model2mass that information. Its dimension should be given by the number of non-flavon particles considered in the model. The index corresponds to the same ordering as the "particles" array. If the index is:
1. zero, then this particle does not belong to any multiplet under the finite discrete group.
2. non-zero value k, then ths particle belongs to a multiplet under the finite discrete group. All the particles with a value of k belong to the same multiplet

Finally, the "fixed_flavon_vevs" key is an array with dimension equal to the number of flavons you included in the "particles" key. Each index corresponds to each flavon. The value for each entry indicates the desired vev. You can choose this convention. The default one is given by:

1. $(1)$ (for singlets)
2. $(1, 0, 0)$
3. $(1, 1, 1)$
4. $(0, 1, 0)$
5. $(0, 0, 1)$
6. $(0, 1, -1)$
7. $(1, \omega^2, \omega)$
8. $(1, \omega, \omega^2)$


where $\omega = \exp\bigg( \frac{2\pi \mathrm{i}}{3} \bigg)$. Of course, the vevs should have a scale. We ignore that here. The other keys in the model file dictionary can be obtained with model2mass. The "param_dict" key was obtained using FlavorPy and gives the $\chi^2$ value in "total_chisq".

In summary, you need to define a dictionary with:
1. "particles" keys which contains the names of the particles in the model.
2. "representations" which contains the irreps under the flavor group.
3. "associations" which contains what are the components of the multiplets for the non-flavon particles.
4. "ZN_symmetries" an array with the dimensions of the $\mathbb{Z}_N$ symmetries considered.
5. "fixed_flavon_vevs" an array with indices that indicate the flavon vev values.

Given these elements in your model dictionary. We need to define more quantities


In [ ]:
particle_list_grouping =[["leftLeptons", "L"], ["rightLeptons", "E"], ["rightNeutrinos", "N"],  ["flavons", "phi"], ["higgsup", "Hu"], ["higgsdown", "Hd"]]


The particle_list_grouping array is used to group the particles in the model in different cattegories. This is useful because then you can tell model2mass which terms in the lagrangian you want to compute:

In [ ]:
allOperators =  [
                        [["rightNeutrinos", "rightNeutrinos"], 1.0],
                        [["flavons", "rightNeutrinos", "rightNeutrinos"], 0.1],
                        [["flavons", "flavons", "rightNeutrinos", "rightNeutrinos"], 0.01],
                        [["rightNeutrinos", "leftLeptons", "higgsup"], 1.0],
                        [["rightNeutrinos", "leftLeptons", "higgsup", "flavons"], 0.1],
                        [["leftLeptons", "rightLeptons", "higgsdown"], 1.0],
                        [["leftLeptons", "rightLeptons", "higgsdown", "flavons"], 0.1]
                    ]

This is telling model2mass that it will compute the lagrangian given by the terms:

$$ \mathcal{L} =  1.0 \alpha^{(1)} NN + 0.1 \alpha^{(2)} \phi N N + 0.01 \alpha^{(3)} \phi \phi N N + 1.0 \alpha^{(4)} N L H_u + 0.1 \alpha^{(5)}N L H_u \phi + 1.0 \alpha^{(6)} L E H_d + 0.1 \alpha^{(7)} L E H_d \phi .$$

Here the $\alpha^{(i)}$ represent coefficients of the different terms. Note that the superscript will not impact the naming of the parameters in the outut of model2mass, as we will see in a bit.

Note that the coefficients in the second element of each array of allOperators denote the supression you want for that term. In this case, we are assuming that the flavon acquires a vev of the form

$$ \phi \sim 0.1 \Lambda $$
where $\Lambda$ is the mass of the right-handed neutrino. Thus, the term $\phi NN $ has a supression of 0.1 with respect to $ NN$. 

Furthermore, we need to specify the number of non_flavon_particles. In this case, it is



In [ ]:
non_flavon_particles = 11

Now, we can initialize our model. We call the Model2Mass class. It needs the following inputs:

1. group: A PyDiscrete group object
2. model_file: A string with the path to your model file (Optional set to None by default)
3. model:  A dictionary with your model as described above (Optional set to None by default)
4. vev_dict:  A dictionary with your vev dictionary as described above (Optional set to the dictionary specified above)
5. non_flavon_particles : An integer that specifies the number of non-flavon particles (Optional set to 11 by default)
6. particle_list_grouping: A list of lists with strings as described above (Optional set to the list specified above)

If you input a model_file, then you do not need a model dictionary. Conversely, if you input a model dictionary, you do not need a model_file.

In [ ]:
model = Model2Mass(groupA4, model = modelByAmber, non_flavon_particles = non_flavon_particles, particle_list_grouping = particle_list_grouping)

We now compute the lagrangian and the list of parameters with get_lagrangian. We set max_terms = 32 such that if the lagrangian has more than 32 terms, then the computation stops.

In [ ]:
lag, params = model.get_lagrangian(allOperators = allOperators, max_terms = 32)

Let's look at the lagrangian

In [ ]:
lag

The params list contains a more organized way of seeing the lagrangian

In [ ]:
params

Let's look at a singlet element

In [ ]:
params[0]

The first element corresponds to the parameter $\alpha_i$ for this lagrangian term. The second element corresponds to the supression it has from the allOperators list. The third element corresponds to the sympy expression that will be added to the lagrangian. The fourth element corresponds to a list of the fields involved. Let's look at the last entry

In [ ]:
params[0][3]

Each element corresponds to a field involved. So this term comes from
$$ \phi_1 \phi_1 N N$$

The flavon $\phi_1$ lives in the third irrep of A4, which corresponds to the $\mathbf{1}''$ irrep (See PyDiscrete tutorial for more details. Then, it transforms with a charge of $4-1 = 3$ under $\mathbb{Z}_4$. A list of all the irreps for all the fields in the model can be obtained from fields_for_discrete

In [ ]:
model.fields_for_discrete(model.model)

After the lagrangian has been computed we can compute the mass matrices using get_mass_matrix. Let's calculate the mass matrix $m_C$ for the charged leptons. For that, we need to specify which are the left fields and which are the right fields in order to compute

$$ m  = \frac{1}{2} \frac{\partial \mathcal{L} }{ \partial L_i \partial R_j} .$$

This package ignores the $1/2$ factors, since we can always rescale the coefficients, and we are only fitting for dimensionless variables. Furthermore, we can use the option factor_out = True to factor one of the \alpha coefficients out.

Moreover, we also need to specify the higgs fields to factor them out.

Finally, get_mass_matrix needs to know the higgs fields involved in the mass term for the Dirac mass cases.

In [ ]:
left_fields = ["leftLeptons", "L"]
right_fields = ["rightLeptons", "E"]
higgs_fields = ["higgsdown", "Hd"]


mC =  model.get_mass_matrix(lag ,params, left_fields = left_fields, right_fields = right_fields, higgs_fields = higgs_fields, factor_out = True)
mC

The right-handed neutrino Dirac mass matrix is quite similar

In [ ]:
left_fields = ["leftLeptons", "L"]
right_fields = ["rightNeutrinos", "N"]
higgs_fields = ["higgsup", "Hu"]


mD =  model.get_mass_matrix(lag ,params, left_fields = left_fields, right_fields = right_fields, higgs_fields = higgs_fields, factor_out = True)
mD

We can also compute Majorana mass matrices using get_mass_matrix. In this case, we just do

In [ ]:
mM =  model.get_mass_matrix(lag ,params, ["rightNeutrinos", "N"], ["rightNeutrinos", "N"])
mM

We can calculate the rank of these three matrices with get_rank_mass_matrix. We will need the sympy matrix from which we will want to calculate its rank and a list of params obtained from get_lagrangian

In [ ]:
mCrank  = model.get_rank_mass_matrix(mC,params)
mDrank  = model.get_rank_mass_matrix(mD,params)
mMrank  = model.get_rank_mass_matrix(mM,params)

print("The rank of mC is: ", mCrank)
print("The rank of mD is: ", mDrank)
print("The rank of mM is: ", mMrank)

We can also obtain a list of  parameters in a mass matrix using ParametersInAMatrix

In [ ]:
model.ParametersInAMatrix(mM)

Finally, we also compute the number of effective flavons, which are the flavons present in the lagrangian, and give a list of the flavons that are not present in the lagrangian

In [ ]:
eff_flavons, flavons_Not_present = model.count_eff_flavons(lag)

print("The number of effective flavons is: ", eff_flavons)
print("The flavons not present in the lagrangian is: ", flavons_Not_present)

The reason why these flavons are not present in the lagrangian is because even though they are charged under the flavor symmetry, there wasn't any flavor invariant that could be constructed from these flavons and the operators given in the allOperators list.


Finally, for completeness, we list all the attributed of the model2mass class.


1. model_file: str. Path to the model_file
2. model: dictionary. Model dictionary with all the necessary specifications to build a lagrangian. See the tutorial for more details.
3. abelian_group_dimensions: array. List with the dimensions of the ZN symmetries. It is obtained from the "ZN_symmetries" key of the model dictionary.
4. parametersSet: dictionary. Dictionary with the values that were set in the model. For instance, the flavon vevs, or the vevs of the higgs to ignore when calculating the dimensionless mass matrix.
5. flavonvevs: list of sympy symbols. Flavon sympy symbols
6. list_of_fields_sympy_symbols: list of sympy symbols. Sympy symbols for all the fields considered
7. list_of_ALL_parameters_in_lag: list of sympy symbols. Sympy symbols for all the paramaeters in the lagrangian (denoted by alpha{n})
8. numberofAbelianSymmetries: int. Number of Abelian Symmetries
9. NumberOfTotalFlavons: int. Number of flavons with a non-zero array in the "representations" key of the model dictionary
10. particle_dict: dictionary. Dictionary with all the particles in a notation digestible for PyDiscrete
11. non_flavon_particles: int. Number of particles that are non-flavon

In [ ]:
print("The model file path is: ", model.model_file) #In this case it is none, since we already opened the file before.

In [ ]:
print("The model dictionary is: ", model.model)

In [ ]:
print("The dimensions of the abelian groups are: ", model.abelian_group_dimensions)

In [ ]:
print("The parameters that were set in the mode are:", model.parametersSet)

In [ ]:
print("The list of flavon vevs are", model.flavonvevs)

In [ ]:
print("The list of sympy symbols of all fields is:", model.list_of_fields_sympy_symbols)

In [ ]:
print("The list of all sympy symbols for the alpha parameters is: ", model.list_of_ALL_parameters_in_lag)

In [ ]:
print("The number of abelian Symmetries is: ", model.numberofAbelianSymmetries)

In [ ]:
print("The number of flavons charged under the flavor symmetry is:", model.NumberOfTotalFlavons)

In [ ]:
print("The dictionary with the particles for PyDiscrete format is:", model.particle_dict)

In [ ]:
print("The number of non flavon particles is:", model.non_flavon_particles)

## Using FlavorPy

For completeness, we now use the package FlavorPy (https://github.com/FlavorPy/FlavorPy/tree/master) to calculate observables. First, we create a list with the strings of all the parameters in the mass matrices

In [ ]:
effparams = model.ParametersInAMatrix(mC) + model.ParametersInAMatrix(mD) + model.ParametersInAMatrix(mM)
effparameters_string = [str(symbol) for symbol in effparams]

We  use sympy lambdify to create a numerical functions from the symbolic mass matrices. Note that we will change our notation from mC to Me since that is the convention in flavorPy. More details in a bit.

In [ ]:
Me_func = sp.lambdify(effparams, mC, modules="numpy")

MD_func = sp.lambdify(effparams, mD, modules="numpy")

MM_func = sp.lambdify(effparams, mM, modules="numpy")



We now define functions that can evaluate the mass matrices for a dictionary of values

In [ ]:
def Me(params_temp): # Charged lepton mass matrix
    listOfvals = []
    for string in effparameters_string:
        globals()[string+'_val'] = params_temp[string]  
        listOfvals.append(globals()[string+'_val'])
    Mematrix = Me_func(*listOfvals)
    
    return (0.1/np.sqrt(3))*np.array(Mematrix).astype(np.complex128)  


def Mn(params_temp): #Neutrino mass matrix after integrating out right-handed neutrinos
    listOfvals = []
    for string in effparameters_string:
        globals()[string+'_val'] = params_temp[string] 
        listOfvals.append(globals()[string+'_val'])
    
    MDmatrix = MD_func(*listOfvals)
    MMmatrix = MM_func(*listOfvals)

    return  np.matmul(np.matmul(np.array(MDmatrix).astype(np.complex128).T,np.linalg.inv(np.array(MMmatrix).astype(np.complex128) )),np.array(MDmatrix).astype(np.complex128) )

We now create the parameter space for FlavorPy

In [ ]:
ParamSpace = mf.ParameterSpace() # This creates a parameter space for flavorpy to fit

# Strings for the parameters in the mass matrices
effparameters_string = [str(symbol) for symbol in model.ParametersInAMatrix(mC) + model.ParametersInAMatrix(mD) + model.ParametersInAMatrix(mM)]

for string in effparameters_string:
    ParamSpace.add_dim(name = string)

We create a flavor model object in FlavorPy

In [ ]:
MyModel = mf.FlavorModel(mass_matrix_e=Me, 
                         mass_matrix_n=Mn, 
                         parameterspace=ParamSpace, 
                         ordering='NO', #Normal Ordering
                         fitted_observables=[ 'me/mu', 'mu/mt', 's12^2', 's13^2', 's23^2', 'd/pi', 'r']) 
# Consider only dimensionless observables

We now call the "param_dict" key from the model file which contains the best obtained in our scans

In [ ]:
mymodel_dict_from_model_file = modelByAmber["param_dict"].copy()

In [ ]:
mymodel_dict_from_model_file

We calculate the $ \chi^2 $

In [ ]:
MyModel.print_chisq(mymodel_dict_from_model_file)

It is the chisq reported in the txt file

In [ ]:
modelByAmber["total_chisq"]

For completeness, we do a quick minimization using FlavorPy. See FlavorPy documentation for details

In [ ]:
df = MyModel.make_fit(points=10, nr_methods=1, max_time=0.000001, retry_time=.1)

In [ ]:
df.head()

A better fit can be found if the fitter runs for sufficcient time.

## LateX table

Model2Mass also has functions to display the model in LaTeX format. Note that these functions are only valid for the conventions in our work (https://arxiv.org/pdf/2506.08080). If you want a different group or a different vev dictionary, you will have to modify these functions.

In [ ]:
from FlavorBuilder.Model2Mass import make_latex_tableA4, make_latex_tableT19

In [ ]:
modelByAmber_file = '../ModelsInPaper/A4XZ4tab2.txt' #Post here your path

In [ ]:
latex_code = make_latex_tableA4(modelByAmber_file)

You can copy and paste the following code for a latex render.

In [ ]:
print(latex_code) 

You can also see the table here

In [ ]:
display(Latex(r'\[' + latex_code + r'\]'))

Similarly for a T19 model

In [ ]:
T19_path = '../ModelsInPaper/T19XZ4tab10.txt'

latex_code = make_latex_tableT19(T19_path)
display(Latex(r'\[' + latex_code + r'\]'))

Recall thet $\omega = \exp \left( \frac{2\pi \mathrm{i}}{3} \right)$